In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [51]:
df = pd.read_csv("act_train.csv")
people = pd.read_csv("people.csv")
print("Tamaño :",df.shape)
print("Tamaño:",people.shape)

Tamaño : (2197291, 15)
Tamaño: (189118, 41)


In [52]:
#Verifiquemos la forma del datdaframe
df.head(2)

,people_id,activity_id,date,activity_category,char_1,char_2,char_3,char_4,char_5,char_6,char_7,char_8,char_9,char_10,outcome
0,ppl_100,act2_1734928,2023-08-26,type 4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 76,0
1,ppl_100,act2_2434093,2022-09-27,type 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,type 1,0


In [53]:
#MIremos cuanta información nos es util.
df.isnull().sum()/df.shape[0] *100
#las columnaas char_1 hasta cchar_9 tienen demasiados datos perdidos. En char_10 se pude interpolar.

people_id             0.000000
activity_id           0.000000
date                  0.000000
activity_category     0.000000
char_1               92.826849
char_2               92.826849
char_3               92.826849
char_4               92.826849
char_5               92.826849
char_6               92.826849
char_7               92.826849
char_8               92.826849
char_9               92.826849
char_10               7.173151
outcome               0.000000
dtype: float64

In [54]:
#Hay que hacer una lista con las columnas que se van a remover.
columnas_a_remover = ["char_"+str(x) for x in np.arange(1,10)]
print("Las columnas que se remueven:",columnas_a_remover)
#Eliminar las columnas
df = df.drop(columnas_a_remover, axis=1)
#Se renobran las columnas para evitar un conflicto al unirse con el otro dataframe.
df = df.rename(columns={"date":"activity_date","char_10":"activity_type"})
#Aquí interpolamos los datos que faltan en la columan char_10 con la moda.
df["activity_type"] = df["activity_type"].fillna(df["activity_type"].mode()[0])
#Miremos la forma de la matriz con los cambios
print("Forma de la matriz:",df.shape)
df.head(2)

Las columnas que se remueven: ['char_1', 'char_2', 'char_3', 'char_4', 'char_5', 'char_6', 'char_7', 'char_8', 'char_9']
Forma de la matriz: (2197291, 6)


,people_id,activity_id,activity_date,activity_category,activity_type,outcome
0,ppl_100,act2_1734928,2023-08-26,type 4,type 76,0
1,ppl_100,act2_2434093,2022-09-27,type 2,type 1,0


In [55]:
#Ahora analicemos los datos de las personas.
people.head(2)

,people_id,char_1,group_1,char_2,date,char_3,char_4,char_5,char_6,char_7,...,char_29,char_30,char_31,char_32,char_33,char_34,char_35,char_36,char_37,char_38
0,ppl_100,type 2,group 17304,type 2,2021-06-29,type 5,type 5,type 5,type 3,type 11,...,False,True,True,False,False,True,True,True,False,36
1,ppl_100002,type 2,group 8688,type 3,2021-01-06,type 28,type 9,type 5,type 3,type 11,...,False,True,True,True,True,True,True,True,False,76


In [56]:
#Igual verifiquemos cuantos datos estan perdidos.
people.isnull().sum().sum()
#Pues no hay nada que interpolar o elimnar. 

0

In [57]:
#Vamos a fucionar ambas matrices.
df = df.merge(people,on=["people_id"],how="inner")
print("Forma de la matriz:", df.shape)

Forma de la matriz: (2197291, 46)


In [58]:
print("Los valores a predecir son :\n",df["outcome"].unique())
print("Como esta distribuian las variables a predecir: \n", df["outcome"].value_counts()/df.shape[0] *100 )

Los valores a predecir son :
 [0 1]
Como esta distribuian las variables a predecir: 
 0    55.60456
1    44.39544
Name: outcome, dtype: float64


In [59]:
#Revisando los tipos de datos con los que trabajaremos
print("Distinct DataTypes:", df.dtypes.unique() ) 

Distinct DataTypes: [dtype('O') dtype('int64') dtype('bool')]


In [60]:
columnas_booleanas = df.select_dtypes("bool")
print( "Nombre de las columnas que se categorizan como 1/0:\n", columnas_booleanas.columns )
#Convertir los valores
for col in columnas_booleanas:
    df[col] = np.where(df[col] == True,1,0)

Nombre de las columnas que se categorizan como 1/0:
 Index(['char_10', 'char_11', 'char_12', 'char_13', 'char_14', 'char_15',
       'char_16', 'char_17', 'char_18', 'char_19', 'char_20', 'char_21',
       'char_22', 'char_23', 'char_24', 'char_25', 'char_26', 'char_27',
       'char_28', 'char_29', 'char_30', 'char_31', 'char_32', 'char_33',
       'char_34', 'char_35', 'char_36', 'char_37'],
      dtype='object')


In [61]:
#Ahora trabajemos con las columnas categoricas
columnas_categoricas = df.select_dtypes("object")
#Revisemos los distintos valores que se deberan recodificar
for col in columnas_categoricas:
    print(col + " column has :", str( len( df[col].unique())) + " distinct values" )

people_id column has : 151295 distinct values
activity_id column has : 2197291 distinct values
activity_date column has : 411 distinct values
activity_category column has : 7 distinct values
activity_type column has : 6515 distinct values
char_1 column has : 2 distinct values
group_1 column has : 29899 distinct values
char_2 column has : 3 distinct values
date column has : 1196 distinct values
char_3 column has : 43 distinct values
char_4 column has : 25 distinct values
char_5 column has : 9 distinct values
char_6 column has : 7 distinct values
char_7 column has : 25 distinct values
char_8 column has : 8 distinct values
char_9 column has : 9 distinct values


In [62]:
#Convertir la variable date de object a datetime. Separar esas fechas por categoria mes, dia, etc..
df["date"] = pd.to_datetime(df["date"])
df["Year"] = df["date"].dt.year
df["Month"] = df["date"].dt.month
df["Quarter"] = df["date"].dt.quarter
df["Week"] = df["date"].dt.week
df["WeekDay"] = df["date"].dt.weekday
df["Day"] = df["date"].dt.day

#Vamos a quitar la columna de fecha por que no nos ayud al momento de entrenar la red.
del(df["date"])
del(df["activity_date"])

C:\Users\EQUIPO\AppData\Local\Temp\ipykernel_10696\4134797014.py:6: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df["Week"] = df["date"].dt.week


  people_id activity_type   activity_id      group_1
0   ppl_100       type 76  act2_1734928  group 17304
1   ppl_100        type 1  act2_2434093  group 17304
2   ppl_100        type 1  act2_3404049  group 17304
3   ppl_100        type 1  act2_3651215  group 17304
4   ppl_100        type 1  act2_4109017  group 17304


In [67]:
#Estas columnas se tienen que depurar para quedarnos con los datos numericos.
print(df[["people_id","activity_type","activity_id", "group_1"]].head())

df.people_id = df.people_id.apply(lambda x:x.split("_")[1])
df.people_id = pd.to_numeric(df.people_id)

df.activity_id = df.activity_id.apply(lambda x:x.split("_")[1])
df.activity_id = pd.to_numeric(df.activity_id)

df.group_1 = df.group_1.apply(lambda x:x.split(" ")[1])
df.group_1 = pd.to_numeric(df.group_1)

df.activity_type = df.activity_type.apply(lambda x:x.split(" ")[1])
df.activity_type = pd.to_numeric(df.activity_type)

print(df[["people_id","activity_type","activity_id", "group_1"]].head())

   people_id  activity_type  activity_id  group_1
0      100.0             76    1734928.0    17304
1      100.0              1    2434093.0    17304
2      100.0              1    3404049.0    17304
3      100.0              1    3651215.0    17304
4      100.0              1    4109017.0    17304
   people_id  activity_type  activity_id  group_1
0      100.0             76    1734928.0    17304
1      100.0              1    2434093.0    17304
2      100.0              1    3404049.0    17304
3      100.0              1    3651215.0    17304
4      100.0              1    4109017.0    17304


In [69]:
#Vamos a codificar las variables categoricas
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
columnas_categoricas = ["activity_category","char_1","char_2","char_3","char_4","char_5","char_6","char_7","char_8","char_9"]

for col in columnas_categoricas:    
    print('\n')
    print('Columns Name: %s' %col)
    print('************')
    df[col] = df[[col]].apply(encoder.fit_transform)
    for i in range(len(encoder.classes_)):
        print(encoder.classes_[i],':', i)

        



Columns Name: activity_category
************
type 1 : 0
type 2 : 1
type 3 : 2
type 4 : 3
type 5 : 4
type 6 : 5
type 7 : 6


Columns Name: char_1
************
type 1 : 0
type 2 : 1


Columns Name: char_2
************
type 1 : 0
type 2 : 1
type 3 : 2


Columns Name: char_3
************
type 1 : 0
type 10 : 1
type 11 : 2
type 12 : 3
type 13 : 4
type 14 : 5
type 15 : 6
type 16 : 7
type 17 : 8
type 18 : 9
type 19 : 10
type 2 : 11
type 20 : 12
type 21 : 13
type 22 : 14
type 23 : 15
type 24 : 16
type 25 : 17
type 26 : 18
type 27 : 19
type 28 : 20
type 29 : 21
type 3 : 22
type 30 : 23
type 31 : 24
type 32 : 25
type 33 : 26
type 34 : 27
type 35 : 28
type 36 : 29
type 37 : 30
type 38 : 31
type 39 : 32
type 4 : 33
type 40 : 34
type 41 : 35
type 42 : 36
type 44 : 37
type 5 : 38
type 6 : 39
type 7 : 40
type 8 : 41
type 9 : 42


Columns Name: char_4
************
type 1 : 0
type 10 : 1
type 11 : 2
type 12 : 3
type 13 : 4
type 14 : 5
type 15 : 6
type 16 : 7
type 17 : 8
type 18 : 9
type 19 : 10
type 

In [74]:
#Separamos la matriz en variables para entrenas y el vecotor a predecir
X = df.loc[:, ['people_id', 'activity_id', 'activity_category', 'activity_type',
       'char_1', 'group_1', 'char_2', 'char_3', 'char_4', 'char_5',
       'char_6', 'char_7', 'char_8', 'char_9', 'char_10', 'char_11', 'char_12',
       'char_13', 'char_14', 'char_15', 'char_16', 'char_17', 'char_18',
       'char_19', 'char_20', 'char_21', 'char_22', 'char_23', 'char_24',
       'char_25', 'char_26', 'char_27', 'char_28', 'char_29', 'char_30',
       'char_31', 'char_32', 'char_33', 'char_34', 'char_35', 'char_36',
       'char_37', 'char_38', 'Year', 'Month', 'Quarter', 'Week', 'WeekDay','Day']] 
y = df.loc[:, ["outcome"]]
print( X.shape, y.shape)

(2197291, 49) (2197291, 1)


In [75]:
#Dividamso los datos para su entremiento, validación y prueba.
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=2018)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=2018)
print("x_train:",x_train.shape)
print("x_test:",x_test.shape)
print("x_val:",x_val.shape)
print("y_train:",y_train.shape)
print("y_test:",y_test.shape)
print("y_val:",y_val.shape)


x_train: (1582048, 49)
x_test: (439459, 49)
x_val: (175784, 49)
y_train: (1582048, 1)
y_test: (439459, 1)
y_val: (175784, 1)


In [77]:
#Ahora si vamos a entrenar la matriz X.
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(256,input_dim=x_train.shape[1],activation="relu"))
model.add(Dense(256,activation="relu"))
model.add(Dense(1,activation="sigmoid")) 
model.compile(optimizer="Adam",loss="binary_crossentropy", metrics=["accuracy"])
model.fit(x_train,y_train,validation_data=(x_val,y_val), epochs=3, batch_size=64)

Epoch 1/3
24720/24720 [==============================] - 213s 8ms/step - loss: 495.5801 - accuracy: 0.6221 - val_loss: 0.6872 - val_accuracy: 0.5545
Epoch 2/3
24720/24720 [==============================] - 210s 8ms/step - loss: 0.6947 - accuracy: 0.5562 - val_loss: 0.6872 - val_accuracy: 0.5545
Epoch 3/3
24720/24720 [==============================] - 206s 8ms/step - loss: 0.6868 - accuracy: 0.5562 - val_loss: 0.6872 - val_accuracy: 0.5545


In [78]:
model = Sequential()
model.add(Dense(256,input_dim = x_train.shape[1],activation="relu"))
model.add(Dense(256,activation="relu"))
model.add(Dense(1,activation="sigmoid"))
model.compile(optimizer="Adam",loss="binary_crossentropy", metrics=["accuracy"])
model.fit(x_train,y_train,validation_data=(x_val,y_val),epochs=2, batch_size=64)

Epoch 1/2
24720/24720 [==============================] - 208s 8ms/step - loss: 419.9440 - accuracy: 0.6220 - val_loss: 0.6866 - val_accuracy: 0.5597
Epoch 2/2
24720/24720 [==============================] - 197s 8ms/step - loss: 0.6919 - accuracy: 0.5562 - val_loss: 0.6902 - val_accuracy: 0.5544


In [79]:
model = Sequential()
model.add(Dense(512,input_dim=x_train.shape[1],activation="relu"))
model.add(Dense(1,activation="sigmoid"))
model.compile(optimizer="Adam",loss="binary_crossentropy", metrics=["accuracy"])
model.fit(x_train,y_train,validation_data=(x_val,y_val),epochs=2, batch_size=64)

Epoch 1/2
24720/24720 [==============================] - 137s 5ms/step - loss: 915.9578 - accuracy: 0.6790 - val_loss: 394.1859 - val_accuracy: 0.6674
Epoch 2/2
24720/24720 [==============================] - 121s 5ms/step - loss: 216.7239 - accuracy: 0.7366 - val_loss: 272.9830 - val_accuracy: 0.5962


In [80]:
model = Sequential()
model.add(Dense(512,input_dim=x_train.shape[1],activation="relu"))
model.add(Dense(512,activation="relu"))
model.add(Dense(1,activation = "sigmoid"))
model.compile(optimizer="Adam",loss="binary_crossentropy",metrics=["accuracy"])
model.fit(x_train,y_train,validation_data=(x_val,y_val),epochs=2, batch_size=64)

Epoch 1/2
24720/24720 [==============================] - 499s 20ms/step - loss: 338.9270 - accuracy: 0.6064 - val_loss: 0.6872 - val_accuracy: 0.5545
Epoch 2/2
24720/24720 [==============================] - 432s 17ms/step - loss: 0.7327 - accuracy: 0.5562 - val_loss: 0.6872 - val_accuracy: 0.5545


In [31]:
#Hay que intentar normalizar, estandarizar o centrar los datos para mirar sihay diferentes resultados.

In [81]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x_train)
x_train_scaled = scaler.transform(x_train)
x_val_scaled = scaler.transform(x_val)
x_test_scaled = scaler.transform(x_test)

In [82]:
model = Sequential()
model.add(Dense(512,input_dim=x_train_scaled.shape[1],activation="relu"))
model.add(Dense(1,activation="sigmoid"))
model.compile(optimizer="Adam",loss="binary_crossentropy",metrics=["accuracy"])
model.fit(x_train_scaled,y_train,validation_data=(x_val_scaled,y_val), epochs=2, batch_size=64)
result = model.evaluate(x_test_scaled,y_test)

for i in range(len(model.metrics_names)):
    print("Metric ",model.metrics_names[i],":", str(round(result[i],2)))

Epoch 1/2
24720/24720 [==============================] - 165s 7ms/step - loss: 0.2788 - accuracy: 0.8668 - val_loss: 0.2454 - val_accuracy: 0.8866
Epoch 2/2
13734/13734 [==============================] - 52s 4ms/step - loss: 0.2123 - accuracy: 0.9067
Metric  loss : 0.21
Metric  accuracy : 0.91


In [83]:
model = Sequential()
model.add(Dense(512,input_dim = x_train_scaled.shape[1],activation="relu"))
model.add(Dense(512,activation="relu"))
model.add(Dense(1,activation="sigmoid"))
model.compile(optimizer="Adam",loss="binary_crossentropy",metrics=["accuracy"])
model.fit(x_train_scaled,y_train,validation_data=(x_val_scaled,y_val),epochs=2, batch_size=64)

Epoch 1/2
24720/24720 [==============================] - 487s 20ms/step - loss: 0.2298 - accuracy: 0.8946 - val_loss: 0.1792 - val_accuracy: 0.9230
Epoch 2/2
24720/24720 [==============================] - 501s 20ms/step - loss: 0.1600 - accuracy: 0.9325 - val_loss: 0.1503 - val_accuracy: 0.9380


In [84]:
model = Sequential()
model.add(Dense(1024,input_dim=x_train_scaled.shape[1],activation="relu"))
model.add(Dense(1024,activation = "relu"))
model.add(Dense(1,activation = "sigmoid"))
model.compile(optimizer = "Adam",loss="binary_crossentropy",metrics=["accuracy"])
model.fit(x_train_scaled,y_train, validation_data=(x_val_scaled,y_val),epochs=2, batch_size=64)

Epoch 1/2
24720/24720 [==============================] - 1712s 69ms/step - loss: 0.2284 - accuracy: 0.8947 - val_loss: 0.1738 - val_accuracy: 0.9251
Epoch 2/2
24720/24720 [==============================] - 2282s 92ms/step - loss: 0.1555 - accuracy: 0.9341 - val_loss: 0.1401 - val_accuracy: 0.9416


In [85]:
model = Sequential()
model.add(Dense(512,input_dim=x_train_scaled.shape[1],activation="relu"))
model.add(Dense(512,activation="relu"))
model.add(Dense(1,activation="sigmoid"))
model.compile(optimizer="Adam",loss="binary_crossentropy",metrics=["accuracy"])
model.fit(x_train_scaled,y_train,validation_data=(x_val_scaled,y_val),epochs=5, batch_size=64)

result = model.evaluate(x_test_scaled,y_test)
for i in range(len(model.metrics_names)):
    print("Metric ",model.metrics_names[i],":", str(round(result[i],2)))

Epoch 1/5
24720/24720 [==============================] - 423s 17ms/step - loss: 0.2283 - accuracy: 0.8950 - val_loss: 0.1764 - val_accuracy: 0.9239
Epoch 2/5
24720/24720 [==============================] - 416s 17ms/step - loss: 0.1584 - accuracy: 0.9328 - val_loss: 0.1526 - val_accuracy: 0.9368
Epoch 3/5
24720/24720 [==============================] - 1466s 59ms/step - loss: 0.1340 - accuracy: 0.9448 - val_loss: 0.1295 - val_accuracy: 0.9469
Epoch 4/5
24720/24720 [==============================] - 484s 20ms/step - loss: 0.1202 - accuracy: 0.9512 - val_loss: 0.1207 - val_accuracy: 0.9510
Epoch 5/5
13734/13734 [==============================] - 103s 7ms/step - loss: 0.1157 - accuracy: 0.9539
Metric  loss : 0.12
Metric  accuracy : 0.95


In [86]:
plt.plot(model.history.history['loss'])
plt.plot(model.history.history['val_loss'])
plt.title("Model's Training & Validation loss across epochs")
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()

KeyError: 'loss'

In [ ]:
plt.plot(model.history.history['acc'])
plt.plot(model.history.history['val_acc'])
plt.title("Model's Training & Validation Accuracy across epochs")
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()